#### 사용자 정의 데이터셋
- Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하 (H/W, S/W) 및 많은 시간 소요에 대한 해결책으로 제시
- 대량 데이터셋 전용 처리 모듈 제공
- Dataset과 DataLoader
    * DataSet       => 데이터셋 전처리, 텐서화 등의 작업 진행
    * DataLoader    => Dataset 인스턴스를 사용해서 배치크기만큼 데이터를 추출함

[1] 모듈 로딩 및 데이터 준비 <hr>

In [5]:
# 모듈로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder      # 타겟 컬럼 수치화

In [6]:
# 데이터
DATA_FILE = r'C:\Users\KDP-27\Desktop\KDT6\MachineLearning\data\iris.csv'

In [7]:
# CSV ==> DtaFrame
irisDF=pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [9]:
# 타겟 컬럼 수치화 ==> LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

irisDF.head(2)



,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


[2] 사용자 정의 데이터셋 클래스 생성 <hr>

In [10]:
# -------------------------------------------------------------------------------------
# 클래스 목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스 이름 : CustomDataSet
# 부모 클래스 : torch.utils.data.Dataset
# 매 개 변 수 : feature, targetDF
# ------------------------------------------------------------------------------------- 

In [37]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()        
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        
        return self.n_rows

    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self,idx):
        fearureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)     # 시리즈-> ndarray -> Tensor 로 변환
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)

        return fearureTS, targetTS

In [38]:
# ----------------------------------------------------------------------------------------
# 함수기능 : 파일 확장자별 데이터프레임 변환 기능
# 함수이름 : convertDataFrame
# 매개변수 : data_path
# 함수결과 : DataFrame
# -----------------------------------------------------------------------------------------

def convertDataFrame(data_path,exit_header=0):
    ext = data_path.rsplit('.')[-1]
    if ext == 'csv':
        return pd.read_csv(data_path, header = exit_header)
    elif ext == 'json':
        return pd.read_json(data_path, header = exit_header)
    elif ext in ['xlsx','xls']:
        return pd.read_excel(data_path, header = exit_header)
    else:
        return pd.read_table(data_path, header = exit_header)



In [40]:
# ----------------------------------------------------------------------------------------
# 클래스기능 : 파일기반 데이터셋
# 클래스이름: FileDataset
# 매개 변수: data_path 파일경로
# 부모클래스 : utils.data.Dataset
# -----------------------------------------------------------------------------------------


class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        super().__init__()        

        # 데이터파일 ==> DataFrame 변환
        dataDF=convertDataFrame(data_path)

        # DataFrame ==> 피쳐와 타겟 추출
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]

        self.n_features=dataDF.shape[1]
        self.n_rows=self.featuresDF.shape[0]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        pass

    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환!!!
    def __getitem__(self,idx):
        fearureTS = torch.FloatTensor(self.featureDF.iloc[idx].values)     # 시리즈-> ndarray -> Tensor 로 변환
        targetTS = torch.FloatTensor(self.targetDF.iloc[idx].values)

        return fearureTS, targetTS


[3] 데이터셋 인스턴스 생성 <hr>

In [18]:
# 피쳐와 타겟 추출
featureDF, targetDF=irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]  
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')




featureDF => (150, 4), targetDF => (150, 1)


In [19]:
# IRIS 데이터셋 인스턴스 생성
irisDS=CustomDataset(featureDF, targetDF)

In [20]:
# IRIS 데이터셋
irisDS.featureDF

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [21]:
irisDS.n_features, irisDS.n_rows

(4, 150)

In [22]:
# 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [24]:
## 필요한 것 : Dataset 인스턴스, Batch_size=1 (기본값)
irisDL=DataLoader(irisDS)

In [26]:
irisDL.__iter__()

In [28]:
for _ in irisDL:
    print(_)
    break

[tensor([[5.1000, 3.5000, 1.4000, 0.2000]]), tensor([[0.]])]
[tensor([[4.9000, 3.0000, 1.4000, 0.2000]]), tensor([[0.]])]
[tensor([[4.7000, 3.2000, 1.3000, 0.2000]]), tensor([[0.]])]
[tensor([[4.6000, 3.1000, 1.5000, 0.2000]]), tensor([[0.]])]
[tensor([[5.0000, 3.6000, 1.4000, 0.2000]]), tensor([[0.]])]
[tensor([[5.4000, 3.9000, 1.7000, 0.4000]]), tensor([[0.]])]
[tensor([[4.6000, 3.4000, 1.4000, 0.3000]]), tensor([[0.]])]
[tensor([[5.0000, 3.4000, 1.5000, 0.2000]]), tensor([[0.]])]
[tensor([[4.4000, 2.9000, 1.4000, 0.2000]]), tensor([[0.]])]
[tensor([[4.9000, 3.1000, 1.5000, 0.1000]]), tensor([[0.]])]
[tensor([[5.4000, 3.7000, 1.5000, 0.2000]]), tensor([[0.]])]
[tensor([[4.8000, 3.4000, 1.6000, 0.2000]]), tensor([[0.]])]
[tensor([[4.8000, 3.0000, 1.4000, 0.1000]]), tensor([[0.]])]
[tensor([[4.3000, 3.0000, 1.1000, 0.1000]]), tensor([[0.]])]
[tensor([[5.8000, 4.0000, 1.2000, 0.2000]]), tensor([[0.]])]
[tensor([[5.7000, 4.4000, 1.5000, 0.4000]]), tensor([[0.]])]
[tensor([[5.4000, 3.9000

In [36]:
## 필요한 것 : Dataset 인스턴스, Batch_size=1 (기본값)
irisDL=DataLoader(irisDS,batch_size=3)

for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)    #batch_size 만큼 출력

torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([3, 4]) torch.Size([3, 1])
torch.Size([